<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 미세 튜닝된 모델 로드 및 사용


이 노트북에는 7장에서 [ch07.ipynb](ch07.ipynb)를 통해 지시 미세 튜닝되어 저장된 모델을 로드하는 최소한의 코드가 포함되어 있습니다.


In [1]:
from importlib.metadata import version

pkgs = [
    "tiktoken",    # 토크나이저
    "torch",       # 딥러닝 라이브러리
]
for p in pkgs:
    print(f"{p} 버전: {version(p)}")

tiktoken version: 0.7.0
torch version: 2.4.0


In [2]:
from pathlib import Path

finetuned_model_path = Path("gpt2-medium355M-sft.pth")
if not finetuned_model_path.exists():
    print(
        f"'{finetuned_model_path}'을(를) 찾을 수 없습니다.\n"
        "`ch07.ipynb` 노트북을 실행하여 모델을 미세 튜닝하고 저장하세요."
    )

In [3]:
from previous_chapters import GPTModel


BASE_CONFIG = {
    "vocab_size": 50257,     # 어휘사전 크기
    "context_length": 1024,  # 문맥 길이
    "drop_rate": 0.0,        # 드롭아웃 비율
    "qkv_bias": True         # 쿼리-키-값 편향
}

model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

CHOOSE_MODEL = "gpt2-medium (355M)"

BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
model = GPTModel(BASE_CONFIG)

In [4]:
import torch

model.load_state_dict(torch.load(
    "gpt2-medium355M-sft.pth",
    map_location=torch.device("cpu"),
    weights_only=True
))
model.eval();

In [5]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [6]:
prompt = """Below is an instruction that describes a task. Write a response 
that appropriately completes the request.

### Instruction:
Convert the active sentence to passive: 'The chef cooks the meal every day.'
"""

In [7]:
from previous_chapters import (
    generate,
    text_to_token_ids,
    token_ids_to_text
)

def extract_response(response_text, input_text):
    return response_text[len(input_text):].replace("### Response:", "").strip()

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids(prompt, tokenizer),
    max_new_tokens=35,
    context_size=BASE_CONFIG["context_length"],
    eos_id=50256
)

response = token_ids_to_text(token_ids, tokenizer)
response = extract_response(response, prompt)
print(response)

The meal is cooked every day by the chef.
